# (4) Training and Validation

In [2]:
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import h5py
import pandas as pd

C:\Users\anrun\anaconda3\envs\EPE_ML\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\anrun\anaconda3\envs\EPE_ML\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\anrun\anaconda3\envs\EPE_ML\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\anrun\anaconda3\envs\EPE_ML\lib\site-packages\tensorflow\pyth

In [2]:
Inputs = Input(shape=(16,))
x = Dense(64, activation='relu', kernel_initializer='lecun_uniform', name='fc1_relu')(Inputs)
y = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name = 'fc2')(x)
z = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name = 'fc3')(y)
a = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name='fc4')(z)
predictions = Dense(5, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'output_sigmoid')(a)
model = Model(inputs=Inputs, outputs=predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
fc1_relu (Dense)             (None, 64)                1088      
_________________________________________________________________
fc2 (Dense)                  (None, 32)                2080      
_________________________________________________________________
fc3 (Dense)                  (None, 32)                1056      
_________________________________________________________________
fc4 (Dense)                  (None, 32)                1056      
_________________________________________________________________
output_sigmoid (Dense)       (None, 5)                 165       
Total params: 5,445
Trainable params: 5,445
Non-trainable params: 0
_________________________________________________________

In [3]:
os.chdir('C:\\Users\\anrun\\Documents\\EPE ML\\Tutorial')
f = h5py.File('processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z', 'r')
treeArray = f['t_allpar_new'][()]

features = ['j_zlogz', 'j_c1_b0_mmdt','j_c1_b1_mmdt', 'j_c1_b2_mmdt' , 'j_c2_b1_mmdt',
            'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 
            'j_n2_b2_mmdt', 'j_mass_mmdt', 'j_multiplicity']
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
features_labels_df = pd.DataFrame(treeArray, columns = features + labels)
features_labels_df = features_labels_df.drop_duplicates()
features_val = features_labels_df[features].values

labels_val = features_labels_df[labels].values

X_train, X_test, y_train, y_test = train_test_split(features_val,labels_val,test_size = 0.2, random_state = 42)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

## Training

In [5]:
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
history = model.fit(X_train, y_train, batch_size = 1024, epochs = 100, 
                    validation_split = 0.25, shuffle = True, callbacks = None,
                    use_multiprocessing=True, workers=4)

Train on 592083 samples, validate on 197361 samples
Epoch 1/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.5440 - accuracy: 0.7957 - val_loss: 0.4370 - val_accuracy: 0.8090
Epoch 2/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.4113 - accuracy: 0.8209 - val_loss: 0.3824 - val_accuracy: 0.8316
Epoch 3/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.3693 - accuracy: 0.8368 - val_loss: 0.3582 - val_accuracy: 0.8427
Epoch 4/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.3533 - accuracy: 0.8449 - val_loss: 0.3469 - val_accuracy: 0.8491
Epoch 5/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.3444 - accuracy: 0.8532 - val_loss: 0.3400 - val_accuracy: 0.8567
Epoch 6/100
592083/592083 [==============================] - 2s 3us/step - loss: 0.3383 - accuracy: 0.8582 - val_loss: 0.3343 - val_accuracy: 0.8597
Epoch 7/100
592083/592083 [===========================

In [ ]:
model.save('4-layer') #Saves to local directory; model file 'two-layer' with no extension

In [ ]:
loaded_model = keras.models.load_model('4-layer') #Loads from local directory

In [ ]:
loaded_model.summary()

## Validation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def learningCurve(history):
    plt.figure(figsize=(10,8))
    plt.plot(history.history['loss'], linewidth=1)
    plt.plot(history.history['val_loss'], linewidth=1)
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['training sample loss','validation sample loss'])
    #plt.savefig('Learning_curve.pdf')
    plt.show()
    plt.close()

In [ ]:
learningCurve(history)

In [ ]:
labels_pred = model.predict(features_val)

In [ ]:
def makeRoc(features_val, labels_val, labels, model, outputDir='', outputSuffix=''):
    from sklearn.metrics import roc_curve, auc
    labels_pred = model.predict(features_val)
    df = pd.DataFrame()
    fpr = {}
    tpr = {}
    auc1 = {}
    plt.figure(figsize=(10,8))       
    for i, label in enumerate(labels):
        df[label] = labels_val[:,i]
        df[label + '_pred'] = labels_pred[:,i]
        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])
        auc1[label] = auc(fpr[label], tpr[label])
        plt.plot(fpr[label],tpr[label],label='%s tagger, AUC = %.1f%%'%(label.replace('j_',''),auc1[label]*100.))
    plt.plot([0, 1], [0, 1], lw=1, color='black', linestyle='--')
    #plt.semilogy()
    plt.xlabel("Background Efficiency")
    plt.ylabel("Signal Efficiency")
    plt.xlim([-0.05, 1.05])
    plt.ylim(0.001,1.05)
    plt.grid(True)
    plt.legend(loc='lower right')
    plt.figtext(0.25, 0.90,'LSTM ROC Curve',fontweight='bold', wrap=True, horizontalalignment='right', fontsize=14)
    #plt.figtext(0.35, 0.90,'preliminary', style='italic', wrap=True, horizontalalignment='center', fontsize=14) 
    #plt.savefig('%sROC_%s.pdf'%(outputDir, outputSuffix))
    return labels_pred

In [ ]:
y_pred = makeRoc(X_test, y_test, labels, model, outputSuffix='two-layer')